In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# Carregar os dados
data = load_breast_cancer()
X, y = data.data, data.target

# Normalizar os dados
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

# Reduzir dimensionalidade para visualização (PCA)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_std)

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.25, random_state=42)

In [4]:
# Função de ativação sigmoide (usada para gerar H — Eq. 2)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [5]:
# Treinamento da ELM (Eq. 9)
def train_elm(X, y, n_hidden=30, lambd=0.1):
    np.random.seed(0)
    n_features = X.shape[1]
    W = np.random.randn(n_features, n_hidden)      # pesos aleatórios
    b = np.random.randn(n_hidden)                  # bias aleatório
    H = sigmoid(X @ W + b)                         # matriz H (Eq. 2})
    A = lambd * np.eye(n_hidden)                   # matriz de regularização (Eq. 5)
    w = np.linalg.inv(H.T @ H + A) @ H.T @ y        # solução analítica (Eq. 9)
    return W, b, w

In [6]:
# Predição da ELM
def predict_elm(X, W, b, w):
    H = sigmoid(X @ W + b)
    y_pred = H @ w
    return (y_pred > 0.5).astype(int)

In [ ]:
lambdas = np.arange(0, 1, 0.01)  # de 0 até 1 com passo 0.05
accuracies = []

for lambd in lambdas:
    W, b, w = train_elm(X_train, y_train, n_hidden=30, lambd=lambd)
    y_pred = predict_elm(X_test, W, b, w)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"λ = {lambd:.3f} → Acurácia = {acc:.4f}")

best_index = np.argmax(accuracies)
best_lambda = lambdas[best_index]
best_accuracy = accuracies[best_index]

print(f"\nMelhor λ = {best_lambda:.3f} com acurácia = {best_accuracy:.4f}")

lambd = best_lambda

In [ ]:
# Treinar e testar
lambd = 1
W, b, w = train_elm(X_train, y_train, n_hidden=30, lambd=lambd)
y_pred = predict_elm(X_test, W, b, w)
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia na base de teste: {acc:.4f}")

In [ ]:
# Plotar a fronteira de decisão com malha menos densa
h = 0.1  # espaçamento maior para evitar MemoryError
x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
grid_points = np.c_[xx.ravel(), yy.ravel()]
Z = predict_elm(grid_points, W, b, w).reshape(xx.shape)

# Fronteira de decisão - ELM com regularização
plt.figure(figsize=(12, 6))
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.3)

# Pontos de treino
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='coolwarm', edgecolor='k', s=40, label='Treino', marker='o')

# Pontos de teste
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='coolwarm', s=40, label='Teste', marker='x')

# Adiciona rótulos personalizados na legenda para as classes 0 e 1
red_patch = mpatches.Patch(color='red', label='Classe 0')
blue_patch = mpatches.Patch(color='blue', label='Classe 1')

# Ajuste final
plt.title(f'Fronteira de Decisão - ELM com Regularização (λ = {lambd})')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.legend(handles=[red_patch, blue_patch], loc='upper right')
plt.grid(True)
plt.show()

In [ ]:
# Plotar a Acurácia em Função de λ
plt.figure(figsize=(8, 5))
plt.plot(lambdas, accuracies, marker='o')
plt.xlabel('λ (Regularização)')
plt.ylabel('Acurácia na base de teste')
plt.title('Acurácia vs λ na ELM')
plt.grid(True)
plt.tight_layout()
plt.show()